# On The Time Evolution of the Heisenberg Model

Hi! 

In this brief nb, I'll present my submission to the IBM Open Science prize and discuss the implementation. The nb includes the following 

- A short demonstration of a Quantum Compiler on Haar random unitaries 
- A depiction of the quantum circuits for Hamiltonian simulation we implemented
- Evaluation of the compiled quantum circuits on IBMQ quantum hardware and noisy simulations, see other experiments nbs to check the full experiments

In the subsection "for submission" below, we run an experiment on the actual Jakarta hardware.  Please run those cells and the all imports below.

The results of our experiments and a brief theoretical background can be found in our accompanying writeup. 

In [1]:
from package import *

In [2]:
#from package.helpers import * Importing python libraries, accessing the IBMQ backends, and simple linear algebra tools
#from package.circuits import * Generating quantum circuits for Hamiltonian Simulation
#from package.compiler import * A Quantum compiler and geometric parsing 
#from package.testing import * Testing scripts

## Importing Backends

We evaluate our quantum circuits on the IBMQ Jakarta, Quito and Manila quantum chips, and their noisy simulations. 

In [3]:
provider1 = IBMQ.load_account()
provider1 = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')
jakarta = provider1.get_backend('ibmq_jakarta')

In [4]:
provider2 = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
quito = provider2.get_backend('ibmq_quito')
manila = provider2.get_backend('ibmq_manila')

In [5]:
noisy_jakarta = QasmSimulator.from_backend(jakarta)
noisy_quito = QasmSimulator.from_backend(quito)
noisy_manila = QasmSimulator.from_backend(manila)

In [6]:
jmap = jakarta.configuration().coupling_map
qmap = quito.configuration().coupling_map
mmap = manila.configuration().coupling_map

In [7]:
# manila and quito have the same set of native gates
basis = jakarta.configuration().basis_gates

# A Quantum Compiler

In order to improve the circuit depth of few-qubit unitaries in our circuits, we implemented a general purpose quantum compiler based on the 'Quantum Shannon Decomposition' (QSD) by Shende et al. This construction parses a general, n qubit unitary into roughly (9/16) 4^n gates, and in particular, any 3-qubit unitary into exactly 24 2-qubit gates configured on a triangle. For details on the method, please refer to the accompanying writeup. 

Note: In terms of n, the compiled circuits require an exponential number of quantum gates, and exponential classical time to compute! This suggests that in the context of optimizing Hamiltonian simulation, these compilers are only useful for small systems, or for optimizing sub-circuits of a larger circuit.

In [8]:
# as a brief demonstration, consider the compilation of n-qubit Haar random unitaries
n = 4
dim = 2**n
U = unitary_group.rvs(dim) # generate the random unitary as a 2D numpy array
qc = gen_qsd(U).decompose() # compile x into a circuit, the gen_qsd(Unitary) function
print("Number of operations", qc.count_ops())

Number of operations OrderedDict([('u', 368), ('cx', 120), ('u3', 72)])


Naturally, the superconducting qubits that these circuits run on at IBMQ are geometrically constrained, and thus we further need to compile these circuits onto such geometries. To do so, we use our geometric_compile function:

In [9]:
instr = geometric_compile(qc, n, jmap) # try jmap, qmap or mmap
qr = QuantumRegister(n)
nqc = QuantumCircuit(qr)
nqc.append(instr, qr)
nqc = nqc.decompose()

print("Number of operations", nqc.count_ops())

Number of operations OrderedDict([('u', 440), ('cx', 246)])


# Generating Quantum Circuits

Here we generate quantum circuit families which perform the time evolution of the XXX Heisenberg model, although they naturally could be extended to any underlying Hamiltonian. 

We perform the following steps to generate our circuits

- generate a quantum circuit for a higher order product formula of the time evolution of the Heisenberg Model
- compile it to a succinct number of 2-qubit gates, using the QSD
- parse it to any underlying circuit geometry, for example, a 1D chain
- transpile it into the native gates and to locally remove redundancies

Ultimately, these ideas will enable us to compile + transpile any 3 qubit gate into 32 2-qubit gates on 1D chain, or 24 2-qubit gates configured on a triangle.  

For instance, the following method generates the 2k-th order product formula of the time evolution of the 3 Qubit Heisenberg model:

In [22]:
nsteps = 8 # number of trotter steps
order = 4 # 2 * k - careful, orders > 6 are slow to compute
time = np.pi

In [23]:
qr, qc = higher_order_trotter(order = order, time = time, trotter_steps = nsteps)
qc.decompose().count_ops()

OrderedDict([('u3', 1539), ('cx', 243)])

In [24]:
# qc.draw()

By compiling our circuit using the Quantum Shannon Decomposition, we can cut down the number of 2-qubit gates for these small sizes to a fixed length. Namely, exactly 24 2-qubit gates on a fully connected circuit layout.

In [25]:
compiled_qc = gen_qsd(Operator(qc).data).decompose()

In [26]:
compiled_qc.count_ops()

OrderedDict([('u', 92), ('cx', 24), ('u3', 12)])

Then, we can compile the circuit down to 1D circuit,  using the long range CNOTs idea discussed in the write-up.

In [27]:
instr = geometric_compile(compiled_qc, 3, line(3))

In [28]:
qr = QuantumRegister(3)
geometric_qc = QuantumCircuit(qr)
geometric_qc.append(instr, qr)
geometric_qc = geometric_qc.decompose()
geometric_qc.count_ops()

OrderedDict([('u', 104), ('cx', 42)])

In [29]:
#geometric_qc.draw()

Finally, lets transpile our geometric circuit to a given set of basis gates, to remove resulting redundancies:

In [30]:
nqc = transpile(geometric_qc, basis_gates = basis, optimization_level = 2)
nqc.count_ops()

OrderedDict([('rz', 90), ('sx', 66), ('cx', 34)])

In [31]:
#nqc.draw()

We can actually confirm these circuits are all the same, up to a global phase, as follows. Given unitaries $U_1$, $U_2$, compute $U_1 U_2^{\dagger}$ and check if the product is a multiple of identity.

Note! After order 6, we start having accuracy issues during transpilation! To adequate we decrease the optimization level.

In [32]:
# given unitaries U1, U2, compute U1 U2^{\dagger} and check if multiple of identity
matrix = np.matmul(Operator(nqc).data, linalg.inv(Operator(geometric_qc).data))
np.allclose(matrix/matrix[0][0] , np.eye(8, 8))

True

Indeed, since we perform all these operations many times on the circuits studied, we have a dedicated function to generate compiled product formula circuits on a chain, following the 3 step procedure above.

In [182]:
_, cqc = compiled_hot(order = 4, time = np.pi,  trotter_steps = 8, basis_gates = basis)

In [183]:
cqc.count_ops()

OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 5)])

# 3 Qubit Experiments

We dedicate this section to simple 3 Qubit experiments, as well as our final submission to the open science prize. We detail our experimental methodology in the accompanying write-up, and use this subsection only for those experiments most related to the prize. In the other nbs in this folder, we present many more experiments.


We test the higher order product formula circuits via the test_formula method, as shown below.  

In [48]:
# inputs: order, number of steps, target time, backend, verbose flag
# try replacing the backend by a noisy simulator, e.g. noisy_jakarta
# if verbose = True, will print circuit information
test_formula(order = 4, trotter_steps = 10, time = np.pi, backend = sim, verbose = False)

state tomography fidelity = 0.9946 ± 0.0008


0.9945533827428168

In [49]:
# a simulation on noisy jakarta
test_formula(order = 4, trotter_steps = 4, time = np.pi, backend = noisy_jakarta, verbose = True)

Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 8)])
global phase: π
                                                                       »
q9726_0: ──────────────────────────────────────────────────────────────»
         ┌──────────────┐     ┌────┐     ┌─────────────┐     ┌────┐    »
q9726_1: ┤ Rz(-0.55237) ├─────┤ √X ├─────┤ Rz(-1.5511) ├─────┤ √X ├────»
         └──────────────┘     └────┘     └─────────────┘     └────┘    »
q9726_2: ──────────────────────────────────────────────────────────────»
              ┌───┐      ┌──────────────┐     ┌────┐    ┌─────────────┐»
q9726_3: ─────┤ X ├──────┤ Rz(-0.59692) ├─────┤ √X ├────┤ Rz(-1.5389) ├»
              └───┘      └──────────────┘     └────┘    └─────────────┘»
q9726_4: ──────────────────────────────────────────────────────────────»
              ┌───┐      ┌──────────────┐                              »
q9726_5: ─────┤ X ├──────┤ Rz(-0.57452) ├──────────────────────────────»
              └───┘      └──

Job ID 46a5a1eb-9a23-4334-ab72-3699c8ec3cd4
Job ID 0492499c-5170-4219-9272-c45b3b590a97
Job ID 68c6b8dd-9fc5-4ee4-a245-8644837005ea
state tomography fidelity = 0.7079 ± 0.0022


0.7078899920552558

To systematically test different number of trotter steps, we use the product_experiment method

In [24]:
product_experiment(np.pi, order = 4, min_steps = 4, max_steps = 20, backend = noisy_jakarta, compiler = True, verbose = True)

4 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 8)])
global phase: π
                                                                       »
q1226_0: ──────────────────────────────────────────────────────────────»
         ┌──────────────┐     ┌────┐     ┌─────────────┐     ┌────┐    »
q1226_1: ┤ Rz(-0.55237) ├─────┤ √X ├─────┤ Rz(-1.5511) ├─────┤ √X ├────»
         └──────────────┘     └────┘     └─────────────┘     └────┘    »
q1226_2: ──────────────────────────────────────────────────────────────»
              ┌───┐      ┌──────────────┐     ┌────┐    ┌─────────────┐»
q1226_3: ─────┤ X ├──────┤ Rz(-0.59692) ├─────┤ √X ├────┤ Rz(-1.5389) ├»
              └───┘      └──────────────┘     └────┘    └─────────────┘»
q1226_4: ──────────────────────────────────────────────────────────────»
              ┌───┐      ┌──────────────┐                              »
q1226_5: ─────┤ X ├──────┤ Rz(-0.57452) ├──────────────────────────────»
     

Job ID 08db4fc9-f5da-4cd7-8c05-c96d977f4f6f
Job ID 7fa8ef7c-d2d9-4fe0-b963-6cec745fb263
Job ID 2a7e1448-b2e2-4ef2-8bb8-1ded893899a9
state tomography fidelity = 0.7118 ± 0.0015
5 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 7)])
global phase: 3.1974e-14
                                                                      »
q1479_0: ─────────────────────────────────────────────────────────────»
         ┌─────────────┐     ┌────┐    ┌──────────────┐     ┌────┐    »
q1479_1: ┤ Rz(-1.6818) ├─────┤ √X ├────┤ Rz(-0.68026) ├─────┤ √X ├────»
         └─────────────┘     └────┘    └──────────────┘     └────┘    »
q1479_2: ─────────────────────────────────────────────────────────────»
              ┌───┐      ┌────────────┐     ┌────┐     ┌─────────────┐»
q1479_3: ─────┤ X ├──────┤ Rz(2.0172) ├─────┤ √X ├─────┤ Rz(-2.7269) ├»
              └───┘      └────────────┘     └────┘     └─────────────┘»
q1479_4: ───────────────────────────────────────

Job ID cfec0c6a-e6dd-4356-b28d-3b653e1f56d0
Job ID 6da28de8-b883-4d44-abab-90d7221b9953
Job ID 25a764d9-efb2-44e4-9bfa-21ab8572ebeb
state tomography fidelity = 0.6769 ± 0.0024
6 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 7)])
global phase: 2π
                                                                      »
q1752_0: ─────────────────────────────────────────────────────────────»
         ┌──────────────┐     ┌────┐    ┌─────────────┐     ┌────┐    »
q1752_1: ┤ Rz(-0.42451) ├─────┤ √X ├────┤ Rz(-2.4016) ├─────┤ √X ├────»
         └──────────────┘     └────┘    └─────────────┘     └────┘    »
q1752_2: ─────────────────────────────────────────────────────────────»
              ┌───┐       ┌────────────┐     ┌────┐    ┌─────────────┐»
q1752_3: ─────┤ X ├───────┤ Rz(1.2111) ├─────┤ √X ├────┤ Rz(-0.3008) ├»
              └───┘       └────────────┘     └────┘    └─────────────┘»
q1752_4: ───────────────────────────────────────────────

Job ID 1b0e0c9a-bc77-47c8-9a4c-cbe884df4a81
Job ID afb43a60-148c-453e-8504-0c9ff7f83443
Job ID 08424f8b-1d4a-4fad-8441-b9996378efc2
state tomography fidelity = 0.6797 ± 0.0017
7 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 9)])
global phase: 2π
                                                                     »
q2045_0: ────────────────────────────────────────────────────────────»
         ┌────────────┐     ┌────┐    ┌─────────────┐     ┌────┐     »
q2045_1: ┤ Rz(2.0327) ├─────┤ √X ├────┤ Rz(-1.6071) ├─────┤ √X ├─────»
         └────────────┘     └────┘    └─────────────┘     └────┘     »
q2045_2: ────────────────────────────────────────────────────────────»
             ┌───┐     ┌─────────────┐     ┌────┐    ┌──────────────┐»
q2045_3: ────┤ X ├─────┤ Rz(-1.2719) ├─────┤ √X ├────┤ Rz(-0.20698) ├»
             └───┘     └─────────────┘     └────┘    └──────────────┘»
q2045_4: ────────────────────────────────────────────────────────

Job ID 8d97f81d-0a3b-4565-bb33-f2026d48fd84
Job ID 96ec2947-f712-45af-a39e-01c464d21aea
Job ID 30eb1eba-8b69-48e4-8cb1-866ba7669b78
state tomography fidelity = 0.6720 ± 0.0008
8 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 7)])
global phase: 2π
                                                                     »
q2358_0: ────────────────────────────────────────────────────────────»
         ┌─────────────┐     ┌────┐    ┌─────────────┐     ┌────┐    »
q2358_1: ┤ Rz(-1.2759) ├─────┤ √X ├────┤ Rz(-1.3548) ├─────┤ √X ├────»
         └─────────────┘     └────┘    └─────────────┘     └────┘    »
q2358_2: ────────────────────────────────────────────────────────────»
              ┌───┐      ┌────────────┐     ┌────┐    ┌─────────────┐»
q2358_3: ─────┤ X ├──────┤ Rz(1.5888) ├─────┤ √X ├────┤ Rz(-3.1099) ├»
              └───┘      └────────────┘     └────┘    └─────────────┘»
q2358_4: ────────────────────────────────────────────────────────

...

Job ID 8af51577-ff46-4c68-8ff7-aef7ed4571dc
Job ID 9cf9f569-cf8c-4457-9900-74464975e85c
Job ID d6b06f1d-bf93-43db-a331-5343d977c772
state tomography fidelity = 0.6701 ± 0.0024
18 Trotterization steps
Circuit Description
OrderedDict([('rz', 73), ('sx', 42), ('cx', 32), ('x', 8)])
global phase: 2π
                                                                     »
q6588_0: ────────────────────────────────────────────────────────────»
         ┌─────────────┐    ┌────┐    ┌────────────┐      ┌────┐     »
q6588_1: ┤ Rz(-1.3013) ├────┤ √X ├────┤ Rz(-1.647) ├──────┤ √X ├─────»
         └─────────────┘    └────┘    └────────────┘      └────┘     »
q6588_2: ────────────────────────────────────────────────────────────»
              ┌───┐     ┌────────────┐    ┌────┐    ┌───────────────┐»
q6588_3: ─────┤ X ├─────┤ Rz(1.5564) ├────┤ √X ├────┤ Rz(-0.011711) ├»
              └───┘     └────────────┘    └────┘    └───────────────┘»
q6588_4: ───────────────────────────────────────────────────────

Job ID 6da826d8-15bd-4df1-88d5-60cb05c5d4e9
Job ID 1f9d259e-f806-4152-940c-7d7661fe910f
Job ID 966d89fa-0696-4559-87e6-0252902f8144
state tomography fidelity = 0.6749 ± 0.0012
19 Trotterization steps
Circuit Description
OrderedDict([('rz', 72), ('sx', 40), ('cx', 32), ('x', 7)])
global phase: 2π
                                                                  »
q7121_0: ─────────────────────────────────────────────────────────»
         ┌──────────┐     ┌────┐    ┌─────────────┐     ┌────┐    »
q7121_1: ┤ Rz(5π/8) ├─────┤ √X ├────┤ Rz(-1.5668) ├─────┤ √X ├────»
         └──────────┘     └────┘    └─────────────┘     └────┘    »
q7121_2: ─────────────────────────────────────────────────────────»
            ┌───┐    ┌─────────────┐     ┌────┐    ┌─────────────┐»
q7121_3: ───┤ X ├────┤ Rz(-1.3997) ├─────┤ √X ├────┤ Rz(-3.1205) ├»
            └───┘    └─────────────┘     └────┘    └─────────────┘»
q7121_4: ─────────────────────────────────────────────────────────»
            ┌───┐    ┌─

Job ID 6a23f1ea-b265-4bae-80c7-bbcde1590914
Job ID 65e43d14-0149-437f-8341-21769bc4bbc7
Job ID 6b1ea236-2c4c-4d3d-9322-f118f1381a76
state tomography fidelity = 0.6758 ± 0.0004
20 Trotterization steps
Circuit Description
OrderedDict([('rz', 73), ('sx', 42), ('cx', 32), ('x', 7)])
global phase: 2π
                                                                    »
q7674_0: ───────────────────────────────────────────────────────────»
         ┌─────────────┐     ┌────┐    ┌─────────────┐    ┌────┐    »
q7674_1: ┤ Rz(-1.3627) ├─────┤ √X ├────┤ Rz(-1.4846) ├────┤ √X ├────»
         └─────────────┘     └────┘    └─────────────┘    └────┘    »
q7674_2: ───────────────────────────────────────────────────────────»
              ┌───┐      ┌────────────┐     ┌────┐    ┌────────────┐»
q7674_3: ─────┤ X ├──────┤ Rz(1.7227) ├─────┤ √X ├────┤ Rz(-3.059) ├»
              └───┘      └────────────┘     └────┘    └────────────┘»
q7674_4: ───────────────────────────────────────────────────────────»
   

Job ID 4f0aae2a-703f-40dc-8913-b75d6c49481f
Job ID 16433295-cd7f-4e01-b2b7-32cd70f5f449
Job ID e88b6d41-999d-4360-a95d-ffd8bd93f754
state tomography fidelity = 0.6712 ± 0.0019


[0.7117523468782547,
 0.6768883166969567,
 0.6796702117151053,
 0.6719896177436061,
 0.6769237421881344,
 0.6668224325090447,
 0.6747642499091459,
 0.6685644808484567,
 0.6711952323740888,
 0.6747446089089316,
 0.6748095026693827,
 0.6695973035471828,
 0.6742926970013338,
 0.6701059917257322,
 0.6749150004090699,
 0.6758336679640043,
 0.6712167651096976]

## For Submission

We dedicate this subsection to our submission to the open science prize. Consider the evaluation of 4 trotter steps of the 4th order product formula, on the Jakarta chip:

last time I ran this, the Job IDs were

625b7b3de1f24579e6eb6d42

625b7b3f9e31154e83838b9b

625b7b42e3b67f7eec3abddd

In [184]:
# please run here
test_formula(order = 4, trotter_steps = 4, time = np.pi, backend = jakarta, verbose = False, compiler = True)

KeyboardInterrupt: 